In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain.chat_models import ChatOpenAI

In [6]:
KEY=os.getenv("OPENAI_API_KEY")

In [7]:
print(KEY)

None


In [8]:
from dotenv import load_dotenv

load_dotenv()

ModuleNotFoundError: No module named 'dotenv'